In [64]:
import pandas as pd

# === Load raw data ===

url = "https://data.cityofchicago.org/api/views/4ijn-s7e5/rows.csv?accessType=DOWNLOAD"
df = pd.read_csv(url)


# === Drop unstructured or irrelevant columns (optional but recommended) ===
df = df.drop(columns=["Violations", "Latitude", "Longitude", "Location"], errors="ignore")

# === Drop rows with missing values in essential fields ===
df = df.dropna()

# === Convert float columns to string for categorical matching (FDs/INDs prefer discrete values) ===
float_cols = df.select_dtypes(include=["float64"]).columns
df[float_cols] = df[float_cols].astype(int).astype(str)

# === Normalize all object columns (trim and lowercase for IND checking) ===
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = df[col].str.strip().str.lower()

# === Export cleaned version for FD/IND analysis ===
df.to_csv("Food_Inspections_Cleaned.csv", index=False)


In [65]:
import pandas as pd
import time
from tane import TANE, read_db

# === Step 1: Load cleaned dataset and prepare sample ===
df = pd.read_csv("Food_Inspections_Cleaned.csv")
columns_to_use = [
    "Inspection ID", "License #", "DBA Name", "Facility Type",
    "Risk", "City", "State", "Results"
]
df_sample = df[columns_to_use]
sample_csv = "Food_Inspections_Sample.csv"
df_sample.to_csv(sample_csv, index=False)

# === Step 2: Load actual column names from saved CSV ===
column_names = pd.read_csv(sample_csv, nrows=0).columns.tolist()

def read_db_true(path, expected_columns=8):
    hashes = {}
    with open(path, 'r') as fin:
        next(fin)  # ✅ Skip header
        for t, line in enumerate(fin):
            line = line.strip()
            if line == '':
                break
            values = line.split(',')[:expected_columns]  # ✅ Cap at 8 columns
            for i, s in enumerate(values):
                hashes.setdefault(i, {}).setdefault(s, set([])).add(t)
        return [PPattern.fix_desc(list(hashes[k].values())) for k in range(expected_columns)]


# === Step 3: Run TANE ===
T = read_db_true(sample_csv, 8)
tane = TANE(T)

start = time.time()
tane.run()
end = time.time()

# === Step 4: Print results directly ===
print(f"\nExecution Time: {end - start:.2f} seconds")
print(f"{len(tane.rules)} FDs found:\n")
print(tane.rules)
for lhs, rhs in tane.rules:
    try:
        lhs_names = [column_names[i] for i in lhs]
        rhs_name = column_names[rhs]
        print(f"{', '.join(lhs_names)} → {rhs_name}")
    except IndexError:
        print(f" Could not map FD: lhs={lhs}, rhs={rhs} — index out of range")



Execution Time: 66.97 seconds
7 FDs found:

[((0,), 1), ((0,), 2), ((0,), 3), ((0,), 4), ((0,), 5), ((0,), 6), ((0,), 7)]
Inspection ID → License #
Inspection ID → DBA Name
Inspection ID → Facility Type
Inspection ID → Risk
Inspection ID → City
Inspection ID → State
Inspection ID → Results


In [66]:
import pandas as pd
import time
from itertools import combinations

def auto_fd_search(df, max_lhs=2):
    results = []
    start = time.time()
    columns = df.columns.tolist()

    for rhs in columns:
        lhs_candidates = [col for col in columns if col != rhs]
        for lhs in combinations(lhs_candidates, max_lhs):
            grouped = df.groupby(list(lhs))[rhs].nunique()
            if grouped.max() == 1:
                results.append((list(lhs), rhs))

    end = time.time()
    print(f"\n max_lhs = {max_lhs} |  Runtime: {end - start:.2f} seconds")
    print(f" {len(results)} functional dependencies found:\n")

    for lhs, rhs in results:
        print(f"{', '.join(lhs)} → {rhs}")
    return results


In [67]:
df = pd.read_csv("Food_Inspections_Sample.csv")

# 1 → 1 FDs
auto_fd_search(df, max_lhs=1)

# 2 → 1 FDs
auto_fd_search(df, max_lhs=2)

# 3 → 1 (optional but slower)
auto_fd_search(df, max_lhs=3)

# 4 → 1 (optional but slower)
auto_fd_search(df, max_lhs=4)



 max_lhs = 1 |  Runtime: 3.23 seconds
 9 functional dependencies found:

Inspection ID → License #
Inspection ID → DBA Name
Inspection ID → Facility Type
Inspection ID → Risk
Inspection ID → City
Inspection ID → State
License # → State
City → State
Inspection ID → Results

 max_lhs = 2 |  Runtime: 16.42 seconds
 52 functional dependencies found:

Inspection ID, DBA Name → License #
Inspection ID, Facility Type → License #
Inspection ID, Risk → License #
Inspection ID, City → License #
Inspection ID, State → License #
Inspection ID, Results → License #
Inspection ID, License # → DBA Name
Inspection ID, Facility Type → DBA Name
Inspection ID, Risk → DBA Name
Inspection ID, City → DBA Name
Inspection ID, State → DBA Name
Inspection ID, Results → DBA Name
Inspection ID, License # → Facility Type
Inspection ID, DBA Name → Facility Type
Inspection ID, Risk → Facility Type
Inspection ID, City → Facility Type
Inspection ID, State → Facility Type
Inspection ID, Results → Facility Type
Inspecti

[(['Inspection ID', 'DBA Name', 'Facility Type', 'Risk'], 'License #'),
 (['Inspection ID', 'DBA Name', 'Facility Type', 'City'], 'License #'),
 (['Inspection ID', 'DBA Name', 'Facility Type', 'State'], 'License #'),
 (['Inspection ID', 'DBA Name', 'Facility Type', 'Results'], 'License #'),
 (['Inspection ID', 'DBA Name', 'Risk', 'City'], 'License #'),
 (['Inspection ID', 'DBA Name', 'Risk', 'State'], 'License #'),
 (['Inspection ID', 'DBA Name', 'Risk', 'Results'], 'License #'),
 (['Inspection ID', 'DBA Name', 'City', 'State'], 'License #'),
 (['Inspection ID', 'DBA Name', 'City', 'Results'], 'License #'),
 (['Inspection ID', 'DBA Name', 'State', 'Results'], 'License #'),
 (['Inspection ID', 'Facility Type', 'Risk', 'City'], 'License #'),
 (['Inspection ID', 'Facility Type', 'Risk', 'State'], 'License #'),
 (['Inspection ID', 'Facility Type', 'Risk', 'Results'], 'License #'),
 (['Inspection ID', 'Facility Type', 'City', 'State'], 'License #'),
 (['Inspection ID', 'Facility Type', 'Cit

In [68]:
import pandas as pd
import time
from itertools import combinations

# Step 1: Brute-force FD discovery
def auto_fd_search(df, max_lhs=2):
    results = []
    columns = df.columns.tolist()
    for rhs in columns:
        lhs_candidates = [col for col in columns if col != rhs]
        for lhs in combinations(lhs_candidates, max_lhs):
            grouped = df.groupby(list(lhs))[rhs].nunique()
            if grouped.max() == 1:
                results.append((list(lhs), rhs))
    return results

# Step 2: Filter for only minimal FDs
def filter_minimal_fds(fds):
    minimal = []
    for lhs, rhs in fds:
        is_minimal = True
        for other_lhs, other_rhs in fds:
            if rhs == other_rhs and set(other_lhs).issubset(set(lhs)) and lhs != other_lhs:
                is_minimal = False
                break
        if is_minimal:
            minimal.append((lhs, rhs))
    return minimal

# Step 3: Run across multiple LHS sizes and combine
def discover_minimal_fds(df, max_lhs=4):
    combined_results = []
    for i in range(1, max_lhs + 1):
        print(f" Searching FDs with max_lhs = {i} ...")
        result = auto_fd_search(df, max_lhs=i)
        combined_results.extend(result)
    minimal = filter_minimal_fds(combined_results)

    print(f"\n Final set of minimal FDs (after pruning redundant ones): {len(minimal)}\n")
    for lhs, rhs in minimal:
        print(f"{', '.join(lhs)} → {rhs}")
    return minimal

df = pd.read_csv("Food_Inspections_Sample.csv")
start_time = time.time()
minimal_fds = discover_minimal_fds(df, max_lhs=4)
total_time = time.time() - start_time

print(f"\n Total Runtime (all steps): {total_time:.2f} seconds")
start_time = time.time()
minimal_fds = discover_minimal_fds(df, max_lhs=3)
total_time = time.time() - start_time

print(f"\n Total Runtime (all steps): {total_time:.2f} seconds")

 Searching FDs with max_lhs = 1 ...
 Searching FDs with max_lhs = 2 ...
 Searching FDs with max_lhs = 3 ...
 Searching FDs with max_lhs = 4 ...

 Final set of minimal FDs (after pruning redundant ones): 11

Inspection ID → License #
Inspection ID → DBA Name
Inspection ID → Facility Type
Inspection ID → Risk
Inspection ID → City
Inspection ID → State
License # → State
City → State
Inspection ID → Results
DBA Name, Facility Type → State
DBA Name, Risk, Results → State

 Total Runtime (all steps): 55.40 seconds
 Searching FDs with max_lhs = 1 ...
 Searching FDs with max_lhs = 2 ...
 Searching FDs with max_lhs = 3 ...

 Final set of minimal FDs (after pruning redundant ones): 11

Inspection ID → License #
Inspection ID → DBA Name
Inspection ID → Facility Type
Inspection ID → Risk
Inspection ID → City
Inspection ID → State
License # → State
City → State
Inspection ID → Results
DBA Name, Facility Type → State
DBA Name, Risk, Results → State

 Total Runtime (all steps): 29.48 seconds


In [69]:
def find_unary_inds_via_index(df, normalize=True, confidence_threshold=1.0):
    value_index = {}
    unary_inds = []

    # Build value index
    for col in df.columns:
        series = df[col].dropna().astype(str)
        if normalize:
            series = series.str.strip().str.lower()
        for val in series.unique():
            value_index.setdefault(val, set()).add(col)

    # Discover unary INDs
    for lhs in df.columns:
        lhs_vals = df[lhs].dropna().astype(str)
        if normalize:
            lhs_vals = lhs_vals.str.strip().str.lower()
        lhs_set = set(lhs_vals)
        total = len(lhs_set)

        if total == 0:
            continue

        for rhs in df.columns:
            if lhs == rhs:
                continue

            match_count = sum(1 for val in lhs_set if rhs in value_index.get(val, set()))
            confidence = match_count / total

            if confidence >= confidence_threshold:
                unary_inds.append({
                    "Subset (LHS)": lhs,
                    "Superset (RHS)": rhs,
                    "Confidence": round(confidence, 4),
                    "Valid": confidence == 1.0
                })

    # Always include all columns even if result is empty
    return pd.DataFrame(unary_inds, columns=["Subset (LHS)", "Superset (RHS)", "Confidence", "Valid"])
df = pd.read_csv("Food_Inspections_Cleaned.csv")

ind_df = find_unary_inds_via_index(df, confidence_threshold=0.95)

if ind_df.empty:
    print(" No INDs found above the confidence threshold.")
else:
    print("Exact INDs (LHS ⊆ RHS):")
    print(ind_df[ind_df["Valid"]])

    print("\n Approximate INDs (LHS ⊆~ RHS):")
    print(ind_df[~ind_df["Valid"]].sort_values(by="Confidence", ascending=False))


 No INDs found above the confidence threshold.


In [70]:
def find_substring_inds(df, normalize=True, confidence_threshold=1.0):
    results = []

    # Preprocess all columns to lists of strings
    processed_cols = {}
    for col in df.columns:
        col_series = df[col].dropna().astype(str)
        if normalize:
            col_series = col_series.str.lower().str.strip()
        processed_cols[col] = col_series.tolist()

    for lhs in df.columns:
        lhs_vals = processed_cols[lhs]
        lhs_unique = list(set(lhs_vals))

        total = len(lhs_unique)
        if total == 0:
            continue

        for rhs in df.columns:
            if lhs == rhs:
                continue

            rhs_values = processed_cols[rhs]

            match_count = 0
            for val in lhs_unique:
                # if val appears as substring in any row of rhs
                if any(val in rhs_val for rhs_val in rhs_values):
                    match_count += 1

            confidence = match_count / total

            if confidence >= confidence_threshold:
                results.append({
                    "Subset (LHS)": lhs,
                    "Superset (RHS)": rhs,
                    "Confidence": round(confidence, 4)s,
                    "Valid": confidence == 1.0
                })

    return pd.DataFrame(results)
df = pd.read_csv("Food_Inspections_Cleaned.csv")

# Substring-based fuzzy IND
substring_ind_df = find_substring_inds(df, confidence_threshold=0.9)

print("Substring-Based INDs (LHS ⊆ RHS):")
print(substring_ind_df[substring_ind_df["Valid"]])

print("\nApproximate Substring-Based INDs:")
print(substring_ind_df[~substring_ind_df["Valid"]].sort_values(by="Confidence", ascending=False))


KeyboardInterrupt: 